In [31]:
from scipy.io import arff
import pandas as pd
import numpy as np

data = arff.loadarff('Wilt_withoutdupl_norm_05.arff')
df = pd.DataFrame(data[0])

df.head()

GLCM_pan  Mean_Green  Mean_Red  Mean_NIR    SD_pan   id   outlier
0  0.656711    0.050984  0.044570  0.218476  0.132110  1.0  b"'yes'"
1  0.680591    0.049425  0.041939  0.177275  0.106749  2.0  b"'yes'"
2  0.734892    0.047396  0.042926  0.259034  0.143741  3.0  b"'yes'"
3  0.698087    0.035317  0.027066  0.127065  0.095697  4.0  b"'yes'"
4  0.738927    0.046076  0.040228  0.295501  0.112481  5.0  b"'yes'"

In [21]:
"""Observe os seguintes:

A dimensão da matriz de covariância inversa é: number_of_features x number_of_features
A dimensão da diffmatriz é semelhante à matriz de dados original:number_of_examples x number_of_features
Assim, cada diff[i](ou seja, linha) é 1 x number_of_features.
Assim, de acordo com a regra de Multiplicação de Matrizes, a matriz resultante diff[i].dot(inv_covariance_matrix)será 1 x 
number_of_featurese quando multiplicarmos novamente, diff[i] numpyconsideraremos automaticamente o posterior como uma matriz 
de coluna, ou seja number_of_features x 1, o resultado final se tornará um valor único! (ou seja, não há necessidade de
transposição)"""

'Observe os seguintes:\n\nA dimensão da matriz de covariância inversa é: number_of_features x number_of_features\nA dimensão da diffmatriz é semelhante à matriz de dados original:number_of_examples x number_of_features\nAssim, cada diff[i](ou seja, linha) é 1 x number_of_features.\nAssim, de acordo com a regra de Multiplicação de Matrizes, a matriz resultante diff[i].dot(inv_covariance_matrix)será 1 x \nnumber_of_featurese quando multiplicarmos novamente, diff[i] numpyconsideraremos automaticamente o posterior como uma matriz \nde coluna, ou seja number_of_features x 1, o resultado final se tornará um valor único! (ou seja, não há necessidade de\ntransposição)'

In [22]:
"""Para detectar os outliers, devemos especificar o limite; nós o fazemos multiplicando a média dos Resultados de Distância
de Mahalanobis pelo grau de exaltação kem que k = 2.0 * stdpara valores extremos e 3.0 * stdpara os valores muito extremos e
isso está de acordo com a regra 68-95-99.7 ( imagem para ilustração do mesmo link ):"""

'Para detectar os outliers, devemos especificar o limite; nós o fazemos multiplicando a média dos Resultados de Distância\nde Mahalanobis pelo grau de exaltação kem que k = 2.0 * stdpara valores extremos e 3.0 * stdpara os valores muito extremos e\nisso está de acordo com a regra 68-95-99.7 ( imagem para ilustração do mesmo link ):'

In [32]:
df['outlier'] = [1 if b== b"'yes'" else 0 for b in df.outlier]

In [33]:
outliers_reais = len(df['outlier'].loc[df['outlier'] == 1])

In [34]:
df.drop(['outlier'], axis=1, inplace=True)

In [35]:
df.drop(['id'], axis=1, inplace=True)

In [36]:
df.dropna().head()

GLCM_pan  Mean_Green  Mean_Red  Mean_NIR    SD_pan
0  0.656711    0.050984  0.044570  0.218476  0.132110
1  0.680591    0.049425  0.041939  0.177275  0.106749
2  0.734892    0.047396  0.042926  0.259034  0.143741
3  0.698087    0.035317  0.027066  0.127065  0.095697
4  0.738927    0.046076  0.040228  0.295501  0.112481

In [37]:
import numpy as np
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

houses = np.array(df)


def MahalanobisDist(data, verbose=False):
    covariance_matrix = np.cov(data, rowvar=False)
    if is_pos_def(covariance_matrix):
        inv_covariance_matrix = np.linalg.inv(covariance_matrix)
        if is_pos_def(inv_covariance_matrix):
            vars_mean = []
            for i in range(data.shape[0]):
                vars_mean.append(list(data.mean(axis=0)))
            diff = data - vars_mean
            md = []
            for i in range(len(diff)):
                md.append(np.sqrt(diff[i].dot(inv_covariance_matrix).dot(diff[i])))

            if verbose:
                print("Covariance Matrix:\n {}\n".format(covariance_matrix))
                print("Inverse of Covariance Matrix:\n {}\n".format(inv_covariance_matrix))
                print("Variables Mean Vector:\n {}\n".format(vars_mean))
                print("Variables - Variables Mean Vector:\n {}\n".format(diff))
                print("Mahalanobis Distance:\n {}\n".format(md))
            return md
        else:
            print("Error: Inverse of Covariance Matrix is not positive definite!")
    else:
        print("Error: Covariance Matrix is not positive definite!")


def MD_detectOutliers(data, extreme=False, verbose=False):
    MD = MahalanobisDist(data, verbose)
    # one popular way to specify the threshold
    #m = np.mean(MD)
    #t = 3. * m if extreme else 2. * m
    #outliers = []
    #for i in range(len(MD)):
    #    if MD[i] > t:
    #        outliers.append(i)  # index of the outlier
    #return np.array(outliers)

    # or according to the 68–95–99.7 rule
    std = np.std(MD)
    k = 3. * std if extreme else 2. * std
    m = np.mean(MD)
    up_t = m + k
    low_t = m - k
    outliers = []
    cont = 0
    for i in range(len(MD)):
        if (MD[i] >= up_t) or (MD[i] <= low_t):
            outliers.append(i)
            cont = cont + 1# index of the outlier
    print(cont)
    return np.array(outliers)


def is_pos_def(A):
    if np.allclose(A, A.T):
        try:
            np.linalg.cholesky(A)
            return True
        except np.linalg.LinAlgError:
            return False
    else:
        return False


data = houses
print("data:\n {}\n".format(data))

outliers_indices = MD_detectOutliers(data, verbose=True)

print("Outliers Indices: {}\n".format(outliers_indices))
print("Outliers:")
cont = 0
for ii in outliers_indices:
    print(data[ii])
    cont = cont + 1
print(cont)

data:
 [[0.65671079 0.0509841  0.04457008 0.21847638 0.13210993]
 [0.68059108 0.04942494 0.04193925 0.17727523 0.10674921]
 [0.73489222 0.04739556 0.04292617 0.25903396 0.14374121]
 ...
 [0.64969377 0.0754982  0.04066498 0.47756716 0.15864998]
 [0.5889552  0.04607564 0.0255317  0.24125758 0.05248847]
 [0.65326883 0.03755116 0.01535408 0.14243164 0.14660091]]

Covariance Matrix:
 [[ 0.00538584 -0.00029351 -0.00028525 -0.00065446 -0.00029844]
 [-0.00029351  0.0013334   0.00141333  0.00102588  0.0005766 ]
 [-0.00028525  0.00141333  0.00160915  0.00068054  0.00063674]
 [-0.00065446  0.00102588  0.00068054  0.01076282  0.00108661]
 [-0.00029844  0.0005766   0.00063674  0.00108661  0.00461244]]

Inverse of Covariance Matrix:
 [[ 1.88788604e+02  5.49603350e+01 -2.02923203e+01  6.86511150e+00
   6.52858850e+00]
 [ 5.49603350e+01  1.34376192e+04 -1.15803716e+04 -5.50521377e+02
   5.20664008e+01]
 [-2.02923203e+01 -1.15803716e+04  1.06528429e+04  4.41953759e+02
  -1.28376473e+02]
 [ 6.86511150e+

In [38]:
outliers_indices

array([ 123,  449,  499,  813,  821,  826,  829,  832,  838,  839,  840,
        844,  859,  864,  867,  871,  888,  901,  959,  967,  970,  973,
        977,  978,  982,  993, 1001, 1007, 1015, 1016, 1018, 1025, 1029,
       1030, 1031, 1032, 1034, 1035, 1036, 1042, 1043, 1045, 1046, 1048,
       1049, 1050, 1052, 1053, 1149, 1240, 2194, 2311, 2313, 2948, 2951,
       3660, 3803, 3805, 3806, 3808, 3819, 3847, 3851, 3852, 3853, 3855,
       3867, 3868, 3944, 3949, 3951, 3952, 3958, 3959, 3961, 3967, 3969,
       3970, 3971, 3972, 3974, 3975, 4020, 4028, 4031, 4032, 4033, 4035,
       4037, 4039, 4041, 4042, 4044, 4045, 4047, 4050, 4051, 4052, 4056,
       4057, 4058, 4060, 4061, 4064, 4065, 4068, 4069, 4070, 4071, 4073,
       4074, 4075, 4079, 4080, 4104, 4108, 4109, 4129, 4130, 4131, 4132,
       4134, 4136, 4137, 4139, 4141, 4151, 4152, 4154, 4155, 4160, 4161,
       4162, 4163, 4168, 4170, 4173, 4174, 4175, 4176, 4177, 4178, 4315,
       4316, 4318, 4322, 4324, 4798, 4801])

In [29]:
print(houses.loc[85])

age          67.0
sex           0.0
cp            2.0
trestbps    115.0
chol        564.0
fbs           0.0
restecg       0.0
thalach     160.0
exang         0.0
oldpeak       1.6
slope         1.0
ca            0.0
thal          3.0
target        1.0
Name: 85, dtype: float64


In [51]:
print(houses.loc[104])

AttributeError: 'numpy.ndarray' object has no attribute 'loc'

In [31]:
print(houses.loc[204])

age          62.0
sex           0.0
cp            0.0
trestbps    160.0
chol        164.0
fbs           0.0
restecg       0.0
thalach     145.0
exang         0.0
oldpeak       6.2
slope         0.0
ca            3.0
thal          3.0
target        0.0
Name: 204, dtype: float64


In [32]:
print(houses.loc[223])

age          56.0
sex           0.0
cp            0.0
trestbps    200.0
chol        288.0
fbs           1.0
restecg       0.0
thalach     133.0
exang         1.0
oldpeak       4.0
slope         0.0
ca            2.0
thal          3.0
target        0.0
Name: 223, dtype: float64


In [33]:
print(houses.loc[266])

age          55.0
sex           0.0
cp            0.0
trestbps    180.0
chol        327.0
fbs           0.0
restecg       2.0
thalach     117.0
exang         1.0
oldpeak       3.4
slope         1.0
ca            0.0
thal          2.0
target        0.0
Name: 266, dtype: float64


In [34]:
print(houses.loc[281])

age          52.0
sex           1.0
cp            0.0
trestbps    128.0
chol        204.0
fbs           1.0
restecg       1.0
thalach     156.0
exang         1.0
oldpeak       1.0
slope         1.0
ca            0.0
thal          0.0
target        0.0
Name: 281, dtype: float64


In [35]:
print(houses.loc[291])

age          58.0
sex           1.0
cp            0.0
trestbps    114.0
chol        318.0
fbs           0.0
restecg       2.0
thalach     140.0
exang         0.0
oldpeak       4.4
slope         0.0
ca            3.0
thal          1.0
target        0.0
Name: 291, dtype: float64


In [36]:
print(houses.loc[297])

age          59.0
sex           1.0
cp            0.0
trestbps    164.0
chol        176.0
fbs           1.0
restecg       0.0
thalach      90.0
exang         0.0
oldpeak       1.0
slope         1.0
ca            2.0
thal          1.0
target        0.0
Name: 297, dtype: float64
